# 連到 Google drive 上的資料夾

In [35]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/CorpusforDDL_compute')
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/CorpusforDDL_compute


# 讀入套件

In [36]:
import re
from collections import Counter
import pickle
import json
import pandas as pd

In [ ]:
# %%capture
# pip install -U ckip-transformers

In [ ]:
# import ckip_transformers
# from ckip_transformers.nlp import CkipWordSegmenter, CkipPosTagger
# ws_driver = CkipWordSegmenter(device=0)
# pos_driver = CkipPosTagger()

# 讀入語料

In [37]:
# 讀入國語日報新聞 txt 檔
mandarin_daily_path = 'CorpusforDDL/國語日報新聞/MandarinDailyNews-20181217'
all_files = os.listdir(mandarin_daily_path)

mandarin_daily_corpus = []
for f in all_files:
  with open(f'{mandarin_daily_path}/{f}') as f:
      lines = f.readlines()
      mandarin_daily_corpus.append(lines)

## 前處理

In [38]:
# 前處理
test = mandarin_daily_corpus[0] # 以 2001.txt 為例
test = [t.replace('\n', '') for t in test] # 移除換行符號
test = [t for t in test if re.findall(r'[\u4e00-\u9fff]+', t)] # 濾掉不含中文字的字串
test[:10]

['陳總統元旦文告調整戒急用忍',
 '【記者夏成淵報導】今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話，並且向全民祝賀新年。陳總統表示，過去政府的「戒急用忍」政策有當時的背景及其必要，但是未來政府將以「積極開放，有效管理」的新視野，規畫我們的新世紀經貿版圖。',
 '陳總統指出，二十一世紀，經濟競爭力依然是我們賴以生存的命脈。面對全球景氣循環的低潮以及包括美國、日本等國家經濟成長的向下修正，臺灣地區的經濟繁榮將再次面臨高難度的考驗，政府與民間要為最壞的可能做最好的準備。',
 '另外，因應我們即將加入世界貿易組織，陳總統說，政府有責任面對全新的經濟情勢，將包括兩岸經貿在內的各項課題，重新納入全球市場的考量。他說，在過去的「戒急用忍」之外，未來我們將以「積極開放，有效管理」的新視野，在知識經濟的既定方針之下，為新世紀經貿版圖做出宏觀的規畫，並且逐步加以落實。',
 '陳水扁總統說，面對過去半年多以來的政經情勢，國人應該都能深刻的體會，有穩定的政局才有經濟的繁榮，有穩定的政局也才有改革和進步。',
 '在兩岸問題方面，陳總統強調，依據中華民國憲法，「一個中國」原本並不是問題，也希望對岸能夠深入了解臺灣地區人民心中的疑慮在哪裡，尊重兩千三百多萬人民的意志。',
 '陳總統認為，兩岸原是一家人，也有共存共榮的相同目標，應該要相互體諒、相互提攜。他呼籲對岸的政府與領導人，尊重中華民國生存的空間與國際的尊嚴，公開放棄武力的威脅，以最大的氣度和前瞻的智慧，超越目前的僵局，從兩岸經貿與文化的統合開始著手，逐步建立兩岸之間的信任，進而共同尋求兩岸永久和平、政治統合的新架構。為二十一世紀兩岸人民最大的福祉，攜手開拓無限可能的空間。',
 '新郵賀吉祥',
 '新歲祈求萬事吉祥，是中國傳統習俗，為讓國人對傳統文化有更深的體認，交通部郵政總局以「並蒂連心」、「連中三元」、「榴開百子」及「白頭富貴」為主題，印製九十年版的「吉祥郵票」，預定明天發行，面值為五元和十二元各兩枚。',
 '國中學力測驗研議加考英聽']

In [ ]:
## 斷詞
# ws  = ws_driver(test)
# pos = pos_driver(ws)
# with open('test_ws.pkl', 'wb') as f:
#     pickle.dump(ws, f)

In [39]:
# 讀入斷詞結果
with open('pkl_files/test_ws.pkl', 'rb') as f:
    test_ws = pickle.load(f)
test_ws = test_ws[:500] # 取前 500 筆為例

In [40]:
# 移除標點符號
def remove_special_characters(sentence):

  pattern = r'[^a-zA-Z0-9\w]'
  cleaned = [re.sub(pattern, '', word) for word in sentence]
  clean_sentence = [word for word in cleaned if len(word) > 0]

  return clean_sentence

In [41]:
test_ws_cleaned = [remove_special_characters(sentence) for sentence in test_ws]
test_ws_cleaned[0]

['陳', '總統', '元旦', '文告', '調整', '戒急用忍']

# 參數計算

## 句子長度
計算方法：算出每一個句子中含有幾個詞彙

In [42]:
sentence_length = [len(sentence) for sentence in test_ws_cleaned]
sentence_length[:10]

[6, 59, 56, 73, 38, 38, 93, 4, 56, 6]

## 詞頻
計算方法：參考 ASBC 詞頻資料，以詞頻 100 為標準，回傳每一個句子中含有幾個高頻詞、幾個低頻詞

In [43]:
# 參考 ASBC 詞頻資料
with open('data/ASBC_unigrams.json', encoding='utf-8') as f:
    word_freq = json.load(f)

cjk = re.compile(r'[\u2E80-\u2FD5\u3190-\u319f\u3400-\u4DBF\u4E00-\u9FCC\uF900-\uFAAD]+')
sorted_wordfreq = sorted(((k, v) for k, v in word_freq.items() if cjk.match(k)), key=lambda x: x[1], reverse=True)

sorted_wordfreq[:10]

[('的', 581859),
 ('是', 149304),
 ('在', 119275),
 ('一', 113236),
 ('有', 81794),
 ('了', 81367),
 ('不', 69865),
 ('我', 69130),
 ('個', 67430),
 ('也', 59640)]

In [44]:
sorted_wordfreq_dict = dict(sorted_wordfreq) # 將詞頻資料化為 dict 方便查詢

def get_high_low_freq(sentence):

  freq = []
  for word in sentence:
    try:
      f = sorted_wordfreq_dict[word]
    except:
      f = 0 # 若是在 sorted_wordfreq_dict 中找不到該詞的頻率資料，則將其頻率指定為 0
    freq.append(f)

  high_low = ['High' if f > 100 else 'Low' for f in freq] # 若在 ASBC 中詞頻大於 100，則視為高頻詞 (High)

  return dict(Counter(high_low))

In [45]:
high_low_freq = [get_high_low_freq(sentence) for sentence in test_ws_cleaned]
high_low_freq[:10]

[{'High': 3, 'Low': 3},
 {'High': 55, 'Low': 4},
 {'High': 52, 'Low': 4},
 {'High': 70, 'Low': 3},
 {'High': 38},
 {'High': 37, 'Low': 1},
 {'High': 85, 'Low': 8},
 {'High': 1, 'Low': 3},
 {'Low': 16, 'High': 40},
 {'High': 3, 'Low': 3}]

## 詞彙等級
計算方法：參考國教院詞語分級表，回傳每一個句子中，各級別的詞彙分別有幾個

In [56]:
# 參考國教院詞語分級表
naer_word_list = pd.read_excel('data/臺灣華語文能力基準詞語表_111-09-20.xlsx')
naer_word_list

,序號,詞語,等別,級別,情境,書面字頻(每百萬字),口語字頻(每百萬字),簡編本系統號,參考注音,參考漢語拼音
0,1,愛,基礎,第1級,核心詞,535,681,"[['愛', ['39542']]]",ㄞˋ,ài
1,2,吧,基礎,第1級,核心詞,706,748,"[['吧', ['32', '103']]]",˙ㄅㄚ,ba
2,3,八,基礎,第1級,核心詞,214,163,"[['八', ['1']]]",ㄅㄚ,bā
3,4,爸爸/爸,基礎,第1級,核心詞,226,806,"[['爸爸', ['82']], ['爸', ['81']]]",ㄅㄚˋ ˙ㄅㄚ / ㄅㄚˋ,bàba / bà
4,5,百,基礎,第1級,核心詞,108,77,"[['百', ['157', '334']]]",ㄅㄞˇ,bǎi
...,...,...,...,...,...,...,...,...,...,...
14462,14463,左右手,精熟,第7級,NaN,3,2,"[['左右手', ['37450']]]",ㄗㄨㄛˇ ㄧㄡˋ ㄕㄡˇ,zuǒ yòu shǒu
14463,14464,坐鎮,精熟,第7級,NaN,5,2,"[['坐鎮', ['37472']]]",ㄗㄨㄛˋ ㄓㄣˋ,zuò zhèn
14464,14465,佐證,精熟,第7級,NaN,5,4,"[['佐證', ['37456']]]",ㄗㄨㄛˇ ㄓㄥˋ,zuǒ zhèng
14465,14466,坐姿,精熟,第7級,NaN,5,1,[],ㄗㄨㄛˋ ㄗ,zuò zī


In [57]:
naer_word_list = naer_word_list.assign(var1=naer_word_list['詞語'].str.split('/')).explode('詞語')
naer_word_list = naer_word_list.explode('var1') # 把用 / 連接的詞語切開做成新的一列
naer_word_list

,序號,詞語,等別,級別,情境,書面字頻(每百萬字),口語字頻(每百萬字),簡編本系統號,參考注音,參考漢語拼音,var1
0,1,愛,基礎,第1級,核心詞,535,681,"[['愛', ['39542']]]",ㄞˋ,ài,愛
1,2,吧,基礎,第1級,核心詞,706,748,"[['吧', ['32', '103']]]",˙ㄅㄚ,ba,吧
2,3,八,基礎,第1級,核心詞,214,163,"[['八', ['1']]]",ㄅㄚ,bā,八
3,4,爸爸/爸,基礎,第1級,核心詞,226,806,"[['爸爸', ['82']], ['爸', ['81']]]",ㄅㄚˋ ˙ㄅㄚ / ㄅㄚˋ,bàba / bà,爸爸
3,4,爸爸/爸,基礎,第1級,核心詞,226,806,"[['爸爸', ['82']], ['爸', ['81']]]",ㄅㄚˋ ˙ㄅㄚ / ㄅㄚˋ,bàba / bà,爸
...,...,...,...,...,...,...,...,...,...,...,...
14462,14463,左右手,精熟,第7級,NaN,3,2,"[['左右手', ['37450']]]",ㄗㄨㄛˇ ㄧㄡˋ ㄕㄡˇ,zuǒ yòu shǒu,左右手
14463,14464,坐鎮,精熟,第7級,NaN,5,2,"[['坐鎮', ['37472']]]",ㄗㄨㄛˋ ㄓㄣˋ,zuò zhèn,坐鎮
14464,14465,佐證,精熟,第7級,NaN,5,4,"[['佐證', ['37456']]]",ㄗㄨㄛˇ ㄓㄥˋ,zuǒ zhèng,佐證
14465,14466,坐姿,精熟,第7級,NaN,5,1,[],ㄗㄨㄛˋ ㄗ,zuò zī,坐姿


In [58]:
#levels = [re.findall(r'\d+\*?', value)[0] for value in naer_word_list['級別'].values]
levels = naer_word_list['級別'].values
word_level_dict = pd.Series(levels, index=naer_word_list['var1']).to_dict()

In [59]:
def get_word_level(sentence):

  levels = []
  for word in sentence:
    try:
      level = word_level_dict[word]
    except:
      level = 'Unknown' # 如果找不到該詞彙，級別設為 Unknown
    levels.append(level)
  
  return dict(Counter(levels))

In [60]:
word_level = [get_word_level(sentence) for sentence in test_ws_cleaned]
word_level[0]

{'Unknown': 3, '第4級': 1, '第4*級': 2}

## 詞彙長度
計算方法：算出每一個句子中有幾個「長度大於三個字」的詞彙

In [61]:
def get_long_word_count(sentence):

  long_word_count = 0
  for word in sentence:
    if len(word) >= 3:
      long_word_count += 1
  
  return long_word_count

In [62]:
long_word_count = [get_long_word_count(sentence) for sentence in test_ws_cleaned] # 算出該句子有幾個「長度大於三個字」的詞彙
long_word_count[:10]

[1, 6, 3, 1, 1, 2, 4, 0, 2, 0]

## 完整的句子
計算方法：用 spacy 取得句子中每一個詞彙的 dependency，若句子同時具備 `nsubj` 和 `dobj`，就視為一個完整的句子

In [63]:
%%capture
!pip install spacy==(last version of spacy(3.2.4)) 
!pip install spacy-transformers
!python -m spacy download zh_core_web_trf

In [64]:
import spacy
import spacy_transformers
nlp = spacy.load('zh_core_web_trf')

In [65]:
# 範例
string = '我 吃 蘋果'
doc = nlp(string)

for token in doc:
    print(token.text, token.pos_, token.dep_)

我 PRON nsubj
吃 VERB ROOT
蘋果 NOUN dobj


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [66]:
test_ws_cleaned_strings = [' '.join(sentence) for sentence in test_ws_cleaned]
test_ws_cleaned_strings[0]

'陳 總統 元旦 文告 調整 戒急用忍'

In [67]:
# %%time
# docs = list(nlp.pipe(test_ws_cleaned_strings[:50], disable=["tok2vec", "tagger", "attribute_ruler", "lemmatizer"]))

In [68]:
%%time
docs = [nlp(x) for x in test_ws_cleaned_strings]

CPU times: user 2min 29s, sys: 8.31 s, total: 2min 37s
Wall time: 2min 41s


In [69]:
def get_tagged_result(tagged_sentence):

  texts = []
  poses = []
  deps = []
  for token in tagged_sentence:
    texts.append(token.text)
    poses.append(token.pos_)
    deps.append(token.dep_)
    
  return texts, poses, deps

In [70]:
tagged_result = [get_tagged_result(d) for d in docs]
tagged_result[0]

(['陳', '總統', '元旦', '文告', '調整', '戒急用忍'],
 ['PROPN', 'NOUN', 'NOUN', 'NOUN', 'VERB', 'NOUN'],
 ['compound:nn', 'nmod:assmod', 'compound:nn', 'nsubj', 'ROOT', 'dobj'])

In [71]:
is_complete_sentence = []
dependencies = ['nsubj', 'dobj']

for text, pos, dep in tagged_result:
  if all(value in dep for value in dependencies):
    ans = 'Y'
  else:
    ans = 'N'
  is_complete_sentence.append(ans)

## 完整語境
計算方式：用 spacy 套件取得句子中每一個詞彙的 pos，再檢查第一個詞彙的 pos 是否為連接詞 `CONJ` `CCONJ` 或副詞 `ADV`

In [72]:
is_complete_context = []

for text, pos, dep in tagged_result:
  if pos[0] not in ['CONJ', 'CCONJ', 'ADV']:
    ans = 'Y'
  else:
    ans = 'N'
  is_complete_context.append(ans)

In [73]:
df = pd.DataFrame({'sentence': test[:500],
                   'sentence_preprocessed': test_ws_cleaned_strings,
                   'sentence_length': sentence_length,
                   'high_low_freq': high_low_freq,
                   'word_level': word_level,
                   'long_word_count': long_word_count,
                   'is_complete_sentence': is_complete_sentence,
                   'is_complete_context': is_complete_context})
df

,sentence,sentence_preprocessed,sentence_length,high_low_freq,word_level,long_word_count,is_complete_sentence,is_complete_context
0,陳總統元旦文告調整戒急用忍,陳 總統 元旦 文告 調整 戒急用忍,6,"{'High': 3, 'Low': 3}","{'Unknown': 3, '第4級': 1, '第4*級': 2}",1,Y,Y
1,【記者夏成淵報導】今天是中華民國九十年元旦，也是二十一世紀的第一天，陳水扁總統發表跨世紀談話...,記者 夏成淵 報導 今天 是 中華民國 九十年 元旦 也 是 二十一世紀 的 第一 天 陳水...,59,"{'High': 55, 'Low': 4}","{'第4級': 12, 'Unknown': 9, '第5級': 9, '第1級': 14,...",6,Y,Y
2,陳總統指出，二十一世紀，經濟競爭力依然是我們賴以生存的命脈。面對全球景氣循環的低潮以及包括美...,陳 總統 指出 二十一世紀 經濟 競爭力 依然 是 我們 賴以 生存 的 命脈 面對 全球 ...,56,"{'High': 52, 'Low': 4}","{'Unknown': 8, '第4級': 8, '第6級': 2, '第1級': 15, ...",3,Y,Y
3,另外，因應我們即將加入世界貿易組織，陳總統說，政府有責任面對全新的經濟情勢，將包括兩岸經貿在...,另外 因應 我們 即將 加入 世界 貿易 組織 陳 總統 說 政府 有 責任 面對 全新 的...,73,"{'High': 70, 'Low': 3}","{'第3級': 2, '第6級': 8, '第1級': 15, '第4級': 13, '第2...",1,Y,N
4,陳水扁總統說，面對過去半年多以來的政經情勢，國人應該都能深刻的體會，有穩定的政局才有經濟的繁...,陳水扁 總統 說 面對 過去 半 年 多 以來 的 政經 情勢 國人 應該 都 能 深刻 的...,38,{'High': 38},"{'Unknown': 6, '第4級': 3, '第1級': 15, '第4*級': 5,...",1,Y,Y
...,...,...,...,...,...,...,...,...
495,【記者劉偉瑩報導】臺北市基督教女青年會舉辦新世紀冬令營活動，內容多采多姿，涵蓋才藝類、成長類...,記者 劉偉瑩 報導 臺北市 基督教 女青年會 舉辦 新 世紀 冬令營 活動 內容 多采多姿 ...,26,"{'High': 19, 'Low': 7}","{'第4級': 5, 'Unknown': 8, '第5級': 2, '第1級': 2, '...",10,Y,Y
496,冬令營活動項目有自然科學啟發營、創意機器人挑戰營、青少年國際觀英語營、東北遊俠單車營，以及好...,冬令營 活動 項目 有 自然 科學 啟發 營 創意 機器人 挑戰 營 青少年 國際觀 英語營...,39,"{'Low': 12, 'High': 27}","{'Unknown': 15, '第2級': 1, '第5級': 6, '第1級': 4, ...",10,N,Y
497,春節親子旅遊現在可安排,春節 親子 旅遊 現在 可 安排,6,{'High': 6},"{'第3級': 1, '第5級': 1, '第4級': 2, '第1級': 1, '第4*級...",0,N,Y
498,【記者劉偉瑩報導】長達八天春節假期，綠生活休閒發展協會推出親子旅遊行程，提供更多選擇，包括合...,記者 劉偉瑩 報導 長達 八 天 春節 假期 綠 生活 休閒 發展 協會 推出 親子 旅遊 ...,31,"{'High': 25, 'Low': 6}","{'第4級': 5, 'Unknown': 9, '第5級': 5, '第1級': 3, '...",3,Y,Y


In [74]:
df.to_csv('CorpusforDDL_parameters_example.csv')